In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from utils import *
from models.scorenetwork import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.ndimage.interpolation import rotate
from itertools import chain
from torchvision import transforms
from torchvision.utils import save_image

from IPython.display import HTML
from IPython.display import clear_output

In [2]:
torch.__version__

'1.8.0'

In [3]:
# initialize random seed
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

cuda = True if torch.cuda.is_available() else False
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
torch.set_default_tensor_type(FloatTensor)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
joint_training = True
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3
sample_dir = './results/MNIST_DAE'
eps = 1e-6
sigma_min = 0.001
sigma_max = 10
n_steps = 10
annealed_step = 100

In [5]:
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../../Data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [6]:
vae = VAE(input_size=image_size, h_dim=h_dim, z_dim=z_dim).to(device)
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)

sn = SN_Model(device, n_steps, sigma_min, sigma_max, dim=z_dim, p = 0.3)
sn_optim = torch.optim.Adam(sn.parameters(), lr = 0.005)
dynamic = AnnealedLangevinDynamic(sigma_min, sigma_max, n_steps, annealed_step, sn, device, eps=eps)

joint_optim = torch.optim.Adam(params=chain(vae.parameters(), sn.parameters()))

# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        x = x.to(device).view(-1, image_size)
        if joint_training:
            mu, log_var = vae.encode(x)
            z = vae.reparameterize(mu, log_var)
            x_reconst = vae.decode(z)
            # Compute reconstruction loss and kl divergence
            reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
            kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            loss_sn = sn.loss_fn(z)
            loss = loss_sn + reconst_loss + kl_div

            joint_optim.zero_grad()
            loss.backward()
            joint_optim.step()
        else:
            #============= First Stage: Update VAE ==============#
            # Forward pass
            x_reconst, mu, log_var = vae(x)
            # Compute reconstruction loss and kl divergence
            # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
            reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
            kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

            # Backprop and optimize
            vae_loss = reconst_loss + kl_div
            vae_optimizer.zero_grad()
            vae_loss.backward()
            vae_optimizer.step()

            #============= Second Stage: Update SN ==============#
            mu, log_var = vae.encode(x)
            z = vae.reparameterize(mu, log_var)

            loss_sn = sn.loss_fn(z)
            vae_optimizer.zero_grad()
            sn_optim.zero_grad()
            loss_sn.backward()
            sn_optim.step()

        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}, Diffuse loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item(), loss_sn.item()))

    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = vae.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = vae(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

        # Save the diffused ima ges
        dynamic = AnnealedLangevinDynamic(sigma_min, sigma_max, n_steps, annealed_step, sn, device, eps=eps)
        sample = dynamic.sampling(batch_size, z_dim, only_final=True)
        out = vae.decode(sample).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'diffuse-{}.png'.format(epoch+1)))


RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasCreate(handle)`

In [8]:
torch.__version__

'1.8.0'